In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pybromo as pbm
print('PyBroMo version:', pbm.__version__)

# Define PSFs

In [ ]:
sx = sy = 0.2e-6; sz = 0.8e-6
psfg = pbm.GaussianPSF(sx=sx, sy=sy, sz=sz)
psfn = pbm.NumericPSF()

In [ ]:
psfg.eval_xz(0.1e-6, 0.1e-6)

In [ ]:
psfn.eval_xz(0.1e-6, 0.1e-6)

In [ ]:
z_peak = psfn.zi[psfn.zm] * 1e-6
z_peak

In [ ]:
dx = dz = 0.01
x = np.arange(-4, 4, dx) * 1e-6
z = np.arange(-6, 6, dz) * 1e-6
X, Z = np.meshgrid(x, z)
X.shape, Z.shape

In [ ]:
idx_z0 = np.abs(z).argmin()
idx_x0 = np.abs(x).argmin()

x[idx_x0], z[idx_z0]

# Evaluate PSFs

In [ ]:
Ig = psfg.eval_xz(X, Z)

plt.figure(figsize=(10, 6))
plt.imshow(Ig.T, extent=(z[0]*1e6, z[-1]*1e6, x[0]*1e6, x[-1]*1e6), origin='lower')
plt.grid(False)
plt.xlabel('μm'); plt.ylabel('μm');

In [ ]:
In = psfn.eval_xz(np.abs(X), Z)

plt.figure(figsize=(10, 60))
plt.imshow(In.T, extent=(z[0]*1e6, z[-1]*1e6, x[0]*1e6, x[-1]*1e6), origin='lower')
plt.grid(False)
plt.xlabel('μm'); plt.ylabel('μm');

In [ ]:
plt.plot((z - z_peak)*1e6, psfn.eval_xz(0, z))
plt.plot(z*1e6, psfg.eval_xz(0, z))
plt.xlabel('Z (μm)');

In [ ]:
plt.plot(x*1e6, psfn.eval_xz(np.abs(x), z_peak))
plt.plot(x*1e6, psfg.eval_xz(x, 0))
plt.xlabel('X (μm)');

In [ ]:
Ig.max(), Ig.min(), Ig.mean()

In [ ]:
In.max(), In.min(), In.mean()

> Both PSF are normalized to have a max of 1. So, depending on the shape they can have different energy (i.e. integrals).
> The timestamp simulation sets the peak emission rate, which is the photon rate detected if a particle would
> stay still in the PSF maximum without diffusing.

# PSF Integrals

Here we compute the PSF integral with different methods.

In [ ]:
from scipy import integrate

In [ ]:
def double_integral(I, x, z, integrate=integrate.simps):
    n = I.shape[1]
    partial_integr_x = np.zeros(n)
    for ix in range(n):
        partial_integr_x[ix] = integrate(I[:, ix], x=z)
        
    # Mirror the x if necessary
    if x[0] == 0:    
        partial_integr_x = np.hstack([partial_integr_x[:0:-1], 
                                      partial_integr_x])
        x = np.hstack([-x[:0:-1], x])
    return integrate(partial_integr_x, x=x)

In [ ]:
def gauss_integral_analytical(sigmas):
    i = 1
    for σ in sigmas:
        i *= np.sqrt(2*np.pi) * σ
    return i

## Gaussian PSF

For Gaussians, we have the analytical form of the integral:

In [ ]:
gauss_integral_analytical(psfg.s[1:]*1e6)

This matches other methods:

In [ ]:
double_integral(Ig, x*1e6, z*1e6)

In [ ]:
integrate.nquad(lambda x, z: psfg.eval_xz(x*1e-6, z*1e-6), [[-4, 4], [-6, 6]])

## Numeric PSF

For the numeric PSF, we integrate on the original data points `hdata` (more accurate):

In [ ]:
double_integral(psfn.hdata, psfn.xi, psfn.zi)

Integrating on resampled data gives similar results:

In [ ]:
double_integral(In, x*1e6, z*1e6)

In [ ]:
# use trapeziod rule instead of Simpson's
double_integral(In, x*1e6, z*1e6, integrate=integrate.trapz)